In [1]:
#%pip install -U "autogen-agentchat"
#%pip install -U "autogen-agentchat[azure]"
#%pip install -U autogenstudio



In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_core.tools import FunctionTool


In [2]:

model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o",
    azure_endpoint="https://aq-aoai-swedencentral.openai.azure.com/",
    model="gpt-4o",
    api_version="2025-03-01-preview",
    api_key="3ShjYyRHd6t0VthIuoOUe1GEnwlj1tkNSHkOs6IrO1CxKQ3gHmsIJQQJ99BAACfhMk5XJ3w3AAABACOG3ddZ",
    temperature=0.7,
)


In [ ]:
#%pip install arxiv


Note: you may need to restart the kernel to use updated packages.


In [3]:
def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:  # type: ignore[type-arg]
    import os
    import time

    import requests
    from bs4 import BeautifulSoup
    from dotenv import load_dotenv

    load_dotenv()

    api_key = os.getenv("GOOGLE_API_KEY")
    search_engine_id = os.getenv("GOOGLE_SEARCH_ENGINE_ID")

    if not api_key or not search_engine_id:
        raise ValueError("API key or Search Engine ID not found in environment variables")

    url = "https://www.googleapis.com/customsearch/v1"
    params = {"key": api_key, "cx": search_engine_id, "q": query, "num": num_results}

    response = requests.get(url, params=params)  # type: ignore[arg-type]

    if response.status_code != 200:
        print(response.json())
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get("items", [])

    def get_page_content(url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            words = text.split()
            content = ""
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += " " + word
            return content.strip()
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return ""

    enriched_results = []
    for item in results:
        body = get_page_content(item["link"])
        enriched_results.append(
            {"title": item["title"], "link": item["link"], "snippet": item["snippet"], "body": body}
        )
        time.sleep(1)  # Be respectful to the servers

    return enriched_results





In [4]:
def arxiv_search(query: str, max_results: int = 2) -> list:  # type: ignore[type-arg]
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv

    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)

    results = []
    for paper in client.results(search):
        results.append(
            {
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "published": paper.published.strftime("%Y-%m-%d"),
                "abstract": paper.summary,
                "pdf_url": paper.pdf_url,
            }
        )

    # # Write results to a file
    # with open('arxiv_search_results.json', 'w') as f:
    #     json.dump(results, f, indent=2)

    return results

In [5]:
google_search_tool = FunctionTool(
    google_search, description="Search Google for information, returns results with a snippet and body content"
)
arxiv_search_tool = FunctionTool(
    arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts"
)


In [6]:
google_search_agent = AssistantAgent(
    name="Google_Search_Agent",
    tools=[google_search_tool],
    model_client=model_client,
    description="An agent that can search Google for information, returns results with a snippet and body content",
    system_message="You are a helpful AI assistant. Solve tasks using your tools.",
)

arxiv_search_agent = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client=model_client,
    description="An agent that can search Arxiv for papers related to a given topic, including abstracts",
    system_message="You are a helpful AI assistant. Solve tasks using your tools. Specifically, you can take into consideration the user's request and craft a search query that is most likely to return relevant academi papers.",
)


report_agent = AssistantAgent(
    name="Report_Agent",
    model_client=model_client,
    description="Generate a report based on a given topic",
    system_message="You are a helpful assistant. Your task is to synthesize data extracted into a high quality literature review including CORRECT references. You MUST write a final report that is formatted as a literature review with CORRECT references.  Your response should end with the word 'TERMINATE'",
)


In [7]:
termination = TextMentionTermination("TERMINATE")
team = RoundRobinGroupChat(
    participants=[google_search_agent, arxiv_search_agent, report_agent], termination_condition=termination
)


In [8]:
await Console(
    team.run_stream(
        task="Write a literature review on no code tools for building multi agent ai systems",
    )
)

await model_client.close()


---------- user ----------
Write a literature review on no code tools for building multi agent ai systems


---------- Google_Search_Agent ----------
[FunctionCall(id='call_C1MjEfv7bfCdB7HXUefr5iRp', arguments='{"query":"literature review on no code tools for building multi-agent AI systems","num_results":5}', name='google_search')]


/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/agents/_assistant_agent.py:870: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


---------- Google_Search_Agent ----------
[FunctionExecutionResult(content='[{\'title\': \'Literature Review — AutoGen\', \'link\': \'https://microsoft.github.io/autogen/dev//user-guide/agentchat-user-guide/examples/literature-review.html\', \'snippet\': \'... literature review on no code tools for building multi agent ai systems", ) ... ### Conclusion No-code tools for building multi-agent AI systems\\\\\\\\xa0\\xa0...\', \'body\': \'Literature Review — AutoGen Skip to main content Back to top Ctrl + K Choose version AgentChat Core Extensions Studio API Reference .NET More 0.2 Docs Search Ctrl + K GitHub Discord Twitter Search Ctrl + K AgentChat Core Extensions Studio API Reference .NET 0.2 Docs GitHub Discord Twitter Installation Quickstart Migration Guide for v0.2 to v0.4 Tutorial Introduction Models Messages Agents Teams Human-in-the-Loop Termination Managing State Advanced Custom Agents Selector Group Chat Swarm\'}, {\'title\': \'Insights and Learnings from Building a Complex Mult